In [3]:
import pandas
import starepandas
import pystare
import netCDF4
import pickle

In [18]:
n_workers = 4

# Load Labels

In [5]:
with open('pickles/largest.pickle', 'rb') as f:
    largest = pickle.load(f)

with open('pickles/labels.pickle', 'rb') as f:
    labels_out = pickle.load(f)
    
with open('pickles/timestamps.pickle', 'rb') as f:
    timestamps = pickle.load(f)

# Load STARE Sidecar

In [6]:
sidecar = netCDF4.Dataset('data/IMERG_stare.nc')['STARE_index'][:].data
sidecar = sidecar.astype('int64')
sidecar = sidecar.T
sidecar = pystare.spatial_coerce_resolution(sidecar, 9)

# Create STAREDF

In [15]:
def make_row(label, timestep):
    sids = sidecar[largest[timestep]==label]
    row = {'sids': sids, 'timestamp' : timestamps[timestep], 'label': label, 'timestep': timestep}    
    return row

label = 1

rows = []
for timestep in range(len(timestamps)):
    row = make_row(label=label, timestep=timestep)
    rows.append(row)

sdf = starepandas.STAREDataFrame(rows, sids='sids')

In [19]:
sdf = sdf.stare_dissolve(by='timestep', n_workers=n_workers).reset_index()

In [20]:
trixels = sdf.make_trixels(n_workers=n_workers)
sdf.set_trixels(trixels, inplace=True)

In [21]:
sdf.head()

,timestep,sids,timestamp,label,trixels
0,0,"[887349864080343048, 887358660173365256, 88737...",2021-01-10 00:00:00,1,"MULTIPOLYGON (((173.10570 22.28556, 173.29270 ..."
1,1,"[887349864080343048, 887358660173365256, 88737...",2021-01-10 00:30:00,1,"MULTIPOLYGON (((173.10570 22.28556, 173.29270 ..."
2,2,"[887349864080343049, 887352063103598601, 88735...",2021-01-10 01:00:00,1,"MULTIPOLYGON (((173.10570 22.28556, 173.19910 ..."
3,3,"[887352063103598601, 887358660173365257, 88736...",2021-01-10 01:30:00,1,"MULTIPOLYGON (((173.29270 22.59424, 173.11849 ..."
4,4,"[885524674778234889, 885537868917768201, 88554...",2021-01-10 02:00:00,1,"MULTIPOLYGON (((177.75804 23.25625, 177.58593 ..."


In [11]:
with open('pickles/featuredb.pickle', 'wb') as f:
    pickle.dump(sdf, f)